# Efficient Yelp API Calls

In [3]:
#install 
!pip install tqdm
!pip install yelpapi

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [7]:
# Credentials and Accessing the API
with open('/Users/kellyji/.secret/yelp_api.json') as f:
    login = json.load(f)
yelp_api = YelpAPI(login['api-key'],timeout_s=5.0)
yelp_api

- API calls to search for sushi restaurants in the Seattle area

In [8]:
LOCATION='Seattle'
TERM='Sushi'

In [10]:
JSON_FILE='/Users/kellyji/Documents/DataEnrichment/Yelp-API-/result_in_progress_SEA_sushi.json'
JSON_FILE

'/Users/kellyji/Documents/DataEnrichment/Yelp-API-/result_in_progress_SEA_sushi.json'

In [13]:
os.path.isfile(JSON_FILE)

False

In [15]:
## Import custom function to create json file
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    file_exists = os.path.isfile(JSON_FILE)
    
    if file_exists == True:
        
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            os.remove(JSON_FILE)
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    else:
        
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        folder = os.path.dirname(JSON_FILE)
        
        if len(folder)>0:
            os.makedirs(folder,exist_ok=True)
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  

In [16]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[i] /Users/kellyji/Documents/DataEnrichment/Yelp-API-/result_in_progress_SEA_sushi.json not found. Saving empty list to new file.
- 0 previous results found.


46

In [17]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/46 [00:00<?, ?it/s]

In [18]:
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,YSQiqH7RIWORk_Qp-A4SOg,shiros-seattle,Shiro's,https://s3-media4.fl.yelpcdn.com/bphoto/hcw9Ea...,False,https://www.yelp.com/biz/shiros-seattle?adjust...,2608,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.5,"{'latitude': 47.6147782162181, 'longitude': -1...","[pickup, delivery]",$$$$,"{'address1': '2401 2nd Ave', 'address2': '', '...",+12064439844,(206) 443-9844,1477.968059
1,3hxncVCRPfmKLOyiXKXbDw,sushi-kashiba-seattle,Sushi Kashiba,https://s3-media2.fl.yelpcdn.com/bphoto/kDUe-m...,False,https://www.yelp.com/biz/sushi-kashiba-seattle...,1403,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.5,"{'latitude': 47.6099251, 'longitude': -122.341...",[delivery],$$$$,"{'address1': '86 Pine St', 'address2': 'Ste 1'...",+12064418844,(206) 441-8844,1781.219913
2,vs5dhED5Yf8kqPp06OD2QQ,momiji-seattle,Momiji,https://s3-media1.fl.yelpcdn.com/bphoto/oQgt6B...,False,https://www.yelp.com/biz/momiji-seattle?adjust...,1826,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.0,"{'latitude': 47.61482, 'longitude': -122.31657}",[delivery],$$,"{'address1': '1522 12th Ave', 'address2': '', ...",+12064574068,(206) 457-4068,1847.275584
3,B62PvT0inodUg6W2VIcGng,the-ivy-seattle,The Ivy,https://s3-media2.fl.yelpcdn.com/bphoto/P9Z2E3...,False,https://www.yelp.com/biz/the-ivy-seattle?adjus...,15,"[{'alias': 'asianfusion', 'title': 'Asian Fusi...",5.0,"{'latitude': 47.6692296, 'longitude': -122.378...",[],NaN,"{'address1': '5512 17th Ave NW', 'address2': '...",,,5845.331678
4,1nHn9R9zntWHfhgICWtL6g,sushi-nori-seattle,Sushi Nori,https://s3-media3.fl.yelpcdn.com/bphoto/nZ5ZTs...,False,https://www.yelp.com/biz/sushi-nori-seattle?ad...,153,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.0,"{'latitude': 47.64177001451144, 'longitude': -...",[],NaN,"{'address1': '2366 Eastlake Ave E', 'address2'...",+12062574213,(206) 257-4213,1962.128929


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
907,mvPyiJwXjcpcZtQgiIIRZA,qfc-bothell-6,QFC,https://s3-media2.fl.yelpcdn.com/bphoto/7ZeT9a...,False,https://www.yelp.com/biz/qfc-bothell-6?adjust_...,41,"[{'alias': 'grocery', 'title': 'Grocery'}, {'a...",3.0,"{'latitude': 47.76518, 'longitude': -122.21164}",[],$,"{'address1': '18921 Bothell Way NE', 'address2...",+14254857548,(425) 485-7548,18083.382518
908,xTRwvTxZHWVUcegd6rcbzg,safeway-renton,Safeway,https://s3-media2.fl.yelpcdn.com/bphoto/09emA7...,False,https://www.yelp.com/biz/safeway-renton?adjust...,92,"[{'alias': 'grocery', 'title': 'Grocery'}]",2.0,"{'latitude': 47.4802287, 'longitude': -122.214...",[],$$,"{'address1': '200 S 3rd St', 'address2': '', '...",+14252554151,(425) 255-4151,18527.952253
909,HYmxD654rewlYLxYP0_TsQ,fred-meyer-bothell,Fred Meyer,https://s3-media3.fl.yelpcdn.com/bphoto/xgELvS...,False,https://www.yelp.com/biz/fred-meyer-bothell?ad...,85,"[{'alias': 'grocery', 'title': 'Grocery'}, {'a...",3.0,"{'latitude': 47.807325, 'longitude': -122.205957}",[],$$,"{'address1': '21045 Bothell Everett Hwy', 'add...",+14253987000,(425) 398-7000,22429.189015
910,xDmeu1Am5zz2oE3Y-ZHHmA,the-cheesecake-factory-alderwood-mall-lynnwood,The Cheesecake Factory Alderwood Mall,https://s3-media2.fl.yelpcdn.com/bphoto/L7eHNL...,False,https://www.yelp.com/biz/the-cheesecake-factor...,656,"[{'alias': 'desserts', 'title': 'Desserts'}, {...",2.5,"{'latitude': 47.8276689, 'longitude': -122.274...",[],$$,"{'address1': '3000 184th St SW', 'address2': '...",+14254121376,(425) 412-1376,22952.787080
911,tj9ycgSpDIxJfp80asdaNg,qfc-mukilteo-2,QFC,https://s3-media3.fl.yelpcdn.com/bphoto/As0SmS...,False,https://www.yelp.com/biz/qfc-mukilteo-2?adjust...,62,"[{'alias': 'grocery', 'title': 'Grocery'}]",2.5,"{'latitude': 47.89122038820605, 'longitude': -...",[],$$,"{'address1': '11700 Mukilteo Speedway', 'addre...",+14252906166,(425) 290-6166,29772.327571


In [19]:
final_df.duplicated(subset='id').sum()

0

In [20]:
# save the final results to a compressed csv
final_df.to_csv('/Users/kellyji/Documents/DataEnrichment/Yelp-API-/misc files/final_result_SEA_sushi.csv.gz', compression='gzip',index=False)